In [ ]:
# to evaluate the error
# negative error never happens
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter
from datetime import datetime as dt
import statistics
from numpy import linspace

DATASETS_DIR = "../tests/input_files/"
# different sets for precision and performance tests
DATASETS =['test_100_25_15', 'test_100_25_25', 'test_100_25_35', 'test_100_25_50',
           'test_100_25_75', 'test_100_50_15', 'test_100_50_25', 'test_100_50_35',
           'test_100_50_50', 'test_100_50_75']
# DATASETS = os.listdir(DATASETS_DIR)
print(DATASETS)

In [ ]:
# run and collect data
CMD_TEMPL = "../CSP {} {} -v 1 -f"
dataset_data = {}
t0 = dt.now()

def get_answer(ans, filename, f=None):
    """Get the smallest program answer."""
    cmd = CMD_TEMPL.format(filename, ans)
    if f:
        cmd += " -z"
    csp_out = subprocess.check_output(cmd, shell=True).decode("utf-8").split("\n")
    if "True" in csp_out:
        # ok, solved
        return ans
    max_cov = 0
    req_cov = 0
    for line in csp_out:
        line_data = line.split()
        if line.startswith("Maximal coverage found is"):
            max_cov = int(line_data[-3])
            req_cov = int(line_data[-1])
            return ans + req_cov - max_cov
            

datasets_num = len(DATASETS)
for num, dataset in enumerate(DATASETS):
    t_d0 = dt.now()
    dataset_metadata = dataset.split("_")
    str_len = int(dataset_metadata[1])
    str_num = int(dataset_metadata[2])
    answer = int(dataset_metadata[3])
    dataset_data[num] = {"answers": [],
                         "answers_f": [],
                         # "answers_b": [],
                         "data": {"str_len": str_len,
                                  "str_num": str_num,
                                  "answer": answer}}
    dataset_dir = os.path.join(DATASETS_DIR, dataset)
    contents = os.listdir(dataset_dir)
    for fnum, fle in enumerate(contents):
        f_path = os.path.join(dataset_dir, fle)
        k = get_answer(answer, f_path)
        k_f = get_answer(answer, f_path, f=True)
        dataset_data[num]["answers"].append(k)
        dataset_data[num]["answers_f"].append(k_f)
        # k_bf = get_from_bruteforce(f_path, answer)
        # dataset_data[num]["answers_b"].append(k_bf)
    print("Dataset {} / {} done in {}".format(num + 1, datasets_num, dt.now() - t_d0))

print("Time spent: {}".format(dt.now() - t0))

In [ ]:
# compare answers with and without F flag
all_answers = []
all_f_answers = []
all_b_answers = []

fig = plt.figure(figsize=(20, 7))
ax_1 = fig.add_subplot(1, 2, 1)

for k, v in dataset_data.items():
    # all_answers.extend(v["answers"])
    # all_f_answers.extend(v["answers_f"])
    # all_b_answers.extend(v["answers_b"])
    ax_1.scatter(v["answers"], v["answers_f"], s=15)

x = linspace(*ax_1.get_xlim())
ax_1.plot(x, x, "--", color="black", alpha=0.5)
ax_1.set_xlabel("Answer without correction")
ax_1.set_ylabel("Answer with correction")
ax_1.grid()

# ax_2 = fig.add_subplot(1, 2, 2)
# x = linspace(*ax_2.get_xlim())
# ax_2.plot(x, x)
# ax_2.set_xlabel("Answer without F flag")
# ax_2.set_ylabel("Answer with bruteforce")
# ax_2.grid()

plt.show()

In [ ]:
# visualize the error
for k, v in dataset_data.items():
    v["errors_f"] = [a - v["data"]["answer"] for a in v["answers_f"]]
    # v["errors_b"] = [a - v["data"]["answer"] for a in v["answers_b"]]
    v["errors"] = [a - v["data"]["answer"] for a in v["answers"]]

ERR_LIM = 50  # ignore errors > 50% for better plotting
fig = plt.figure(figsize=(20, 10))
to_plot = []
titles = []

perc_errors = []
perc_errors_f = []
perc_errors_b = []
answers = []

for num, (k, v) in enumerate(dataset_data.items(), 1):
    # ax = fig.add_subplot(rows, cols, num)
    ans = v["data"]["answer"]
    ans_arr = [ans for _ in range(len(v["errors"]))]
    answers.extend(ans_arr)

    perc_error = [e / ans * 100 for e in v["errors"]]
    perc_error_f = [e / ans * 100 for e in v["errors_f"]]
    # perc_error_b = [e / ans * 100 for e in v["errors_b"]]
    
    perc_error = [e if e < ERR_LIM else ERR_LIM for e in perc_error]
    perc_error_f = [e if e < ERR_LIM else ERR_LIM for e in perc_error_f]
    # perc_error_b = [e if e < ERR_LIM else ERR_LIM for e in perc_error_b]

    perc_errors.extend(perc_error)
    perc_errors_f.extend(perc_error_f)
    # perc_errors_b.extend(perc_error_b)

    to_plot.append(perc_error)
    to_plot.append(perc_error_f)
    # to_plot.append(perc_error_b)

#     to_plot.append(v["errors"])
#     to_plot.append(v["errors_f"])
    # to_plot.append(v["errors_b"])

    title = "{} {} {}".format(v["data"]["str_len"], v["data"]["str_num"], ans)
    titles.append(title)
    titles.append(title + "corr")
    # titles.append(title + "B")
    # ax.set_title(title)
ax_1 = fig.add_subplot(1, 1, 1)

pos = list(range(1, len(titles) + 1))
vp = ax_1.violinplot(to_plot, pos,
                     showmeans=True,
                     showextrema=True,
                     showmedians=True)

ax_1.set_title("Errors with and without a correction")
ax_1.set_ylabel("Error %")

ax_1.set_xticks(pos)
ax_1.set_xticklabels(titles, rotation=45, fontsize=8)
ax_1.grid(axis="y")

colors = ["navy", "navy",
          "purple", "purple",
          "orange", "orange",
          "green", "green",
          "pink", "pink",
          ] * 2
vp["cmeans"].set_edgecolor("blue")
vp["cmedians"].set_edgecolor("green")
vp["cbars"].set_edgecolor("grey")
vp["cmaxes"].set_edgecolor("grey")
vp["cmins"].set_edgecolor("grey")

for vb, color in zip(vp['bodies'], colors):
    vb.set_facecolor(color)
    vb.set_edgecolor("grey")
plt.show()

print(vp.keys())

print("Mean uncorr error: ", statistics.mean(perc_errors))
print("Median uncorr error: ", statistics.median(perc_errors))

print("Mean corr error: ", statistics.mean(perc_errors_f))
print("Median corr error: ", statistics.median(perc_errors_f))

In [ ]:
no_err = perc_errors_f.count(0)
print(f"No errors: {no_err} measurements")
print(f"Errors in: {len(perc_errors_f) - no_err} measurements")